In [16]:
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

In [17]:
dataset = pd.read_csv(r"V:\Krishna\College\Project\fipe_cars.csv")
# dataset.head()
dataset.isnull().sum()

year_of_reference     0
month_of_reference    0
fipe_code             0
authentication        0
brand                 0
model                 0
fuel                  0
gear                  0
engine_size           0
year_model            0
avg_price_brl         0
dtype: int64

In [18]:
X = dataset.drop(columns=["year_of_reference","month_of_reference","fipe_code","authentication","model","avg_price_brl"],axis=1)
Y = dataset["avg_price_brl"]
# X["brand"].value_counts()
# X["engine_size"].value_counts()
# X["year_model"].value_counts().head(25)

In [19]:
encoder = LabelEncoder()
X["fuel"] = encoder.fit_transform(X["fuel"])
X["brand"] = encoder.fit_transform(X["brand"])
X["gear"] =  encoder.fit_transform(X["gear"])
# X["year_model"] = encoder.fit_transform(X["year_model"])
# X["engine_size"].value_counts()
# X["year_model"].value_counts().head(25)
X.head()

,brand,fuel,gear,engine_size,year_model
0,0,2,1,1.5,2005
1,0,0,1,1.5,2005
2,0,0,1,1.5,2004
3,0,0,1,1.5,2003
4,0,2,1,1.5,2002


In [20]:
scale = StandardScaler()
X = scale.fit_transform(X)

In [21]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3,random_state=42)
X_train.shape,X_test.shape

((141262, 5), (60542, 5))

In [22]:
# linreg = LinearRegression()
# linreg.fit(X_train,Y_train)

# prediction = linreg.predict(X_test)
# metrics.r2_score(Y_test,prediction)

In [23]:
# lasreg = Lasso()
# lasreg.fit(X_train,Y_train)

# prediction = lasreg.predict(X_test)
# metrics.r2_score(Y_test,prediction)

In [24]:
randfor = RandomForestRegressor(n_estimators = 100)
randfor.fit(X_train,Y_train)

prediction = randfor.predict(X_test)
metrics.r2_score(Y_test,prediction)*100

94.41018359842556

In [25]:
# supvec = SVR(kernel='linear')
# supvec.fit(X_train,Y_train)

# prediction = supvec.predict(X_test)
# metrics.r2_score(Y_test,prediction)

In [26]:
# ridreg = Ridge(alpha=1)
# ridreg.fit(X_train,Y_train)

# prediction = ridreg.predict(X_test)
# metrics.r2_score(Y_test,prediction)

In [27]:
# with open("V:\Krishna\College\Project\Completed\Car_Price_model3.pkl", "wb") as model_file:
#     pickle.dump(randfor, model_file)

In [28]:
import pandas as pd
import pickle
import tkinter as tk
from tkinter import messagebox

# Load the machine learning model
model = pickle.load(open("V:\Krishna\College\Project\Car_Price_model3.pkl", "rb"))

def predict_price():
    try:
        # Converting Fuel Type
        temp = fuel_type_var.get()
        if temp == "Diesel":
            fuel_type = 1
        elif temp == "Petrol":
            fuel_type = 2
        elif temp == "LPG":
            fuel_type = 0

        # Converting Transmission Type
        temp = transmission_type_var.get()
        if temp == "Manual":
            transmission_type = 1
        else:
            transmission_type = 0
            
        # Converting The Brand
        manufacturer = manufacturer_var.get()
        manufacturer_mapping = {
            "Maruti Suzuki": 5,
            "Volkswagen": 7,
            "Hyundai": 2,
            "Honda": 1,
            "Toyota": 6,
            "Fiat": 0,
            "Renault": 4,
            "Mahindra": 3
        }
        Manufacturer = manufacturer_mapping.get(manufacturer, 0)

        # Converting Year
        year = year_var.get()

        year_mapping = {
            2022: 37,
            2021: 36,
            2020: 35,
            2019: 34,
            2018: 33,
            2017: 32,
            2016: 31,
            2015: 30,
            2014: 29,
            2013: 28,
            2012: 27,
            2011: 26,
            2010: 25,
            2009: 24,
            2008: 23
        }
        year = year_mapping.get(year, 0)

        Engine = engine_entry.get()
        Engine = 4.5 - float(Engine)

        input_data = pd.DataFrame({
            "brand": [Manufacturer],
            "year_model": [year],
            "fuel": [fuel_type],
            "gear": [transmission_type],
            "engine_size": [Engine],
        })

        predicted_price = model.predict(input_data)[0]
        predicted_price = abs(predicted_price * 10) / 150000
        predicted_price_label.config(text=f"Predicted Price: {predicted_price:.2f} lakhs")
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred: {e}")

# Create the main application window
root = tk.Tk()
root.title("Car Price Prediction")
root.geometry("400x400")  # Adjust the window size

# Create a frame for better organization
frame = tk.Frame(root)
frame.pack(padx=20, pady=20)

# Manufacturer
manufacturer_label = tk.Label(frame, text="Brand:")
manufacturer_label.grid(row=0, column=0, padx=10, pady=5)
manufacturer_var = tk.StringVar(root)
manufacturer_var.set("Volkswagen")
manufacturer_option = tk.OptionMenu(frame, manufacturer_var, "Maruti Suzuki", "Volkswagen", "Hyundai", "Honda",
                                    "Toyota", "Fiat", "Renault", "Mahindra")
manufacturer_option.grid(row=0, column=1, padx=10, pady=5)

# Year
year_label = tk.Label(frame, text="Year:")
year_label.grid(row=1, column=0, padx=10, pady=5)
year_var = tk.IntVar(root)
year_var.set(2015)
year_option = tk.OptionMenu(frame, year_var, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022)
year_option.grid(row=1, column=1, padx=10, pady=5)

# Fuel type
fuel_type_label = tk.Label(frame, text="Fuel Type:")
fuel_type_label.grid(row=2, column=0, padx=10, pady=5)
fuel_type_var = tk.StringVar(root)
fuel_type_var.set("Petrol")
fuel_type_option = tk.OptionMenu(frame, fuel_type_var, "Petrol", "Diesel", "LPG")
fuel_type_option.grid(row=2, column=1, padx=10, pady=5)

# Transmission
transmission_type_label = tk.Label(frame, text="Transmission Type:")
transmission_type_label.grid(row=3, column=0, padx=10, pady=5)
transmission_type_var = tk.StringVar(root)
transmission_type_var.set("Manual")
transmission_type_option = tk.OptionMenu(frame, transmission_type_var, "Manual", "Automatic")
transmission_type_option.grid(row=3, column=1, padx=10, pady=5)

# Engine CC
engine_label = tk.Label(frame, text="Engine Size Based on Liters:")
engine_label.grid(row=4, column=0, padx=10, pady=5)
engine_entry = tk.Entry(frame)
engine_entry.grid(row=4, column=1, padx=10, pady=5)

# Button
predict_button = tk.Button(root, text="Predict Price", command=predict_price)
predict_button.pack()

predicted_price_label = tk.Label(root, text="")
predicted_price_label.pack()

root.mainloop()
